In [ ]:
import psycopg2
import pandas as pd
from psycopg2 import OperationalError
import pickle
from io import BytesIO
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os
from dotenv import load_dotenv

load_dotenv()

# Leer las variables de entorno
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')


## Funciones ha utilizar

In [3]:
def do_query(query):
    try:
        # Crear un cursor
        conn = psycopg2.connect(
                dbname="punto_seguro",
                user=DB_USER,
                password=DB_PASSWORD,
                host="pg-ed88bff-srodriguezcabana-914d.h.aivencloud.com",
                port="23178"
            )
        conn.autocommit = True
        cursor = conn.cursor()

        df_resultados = pd.read_sql_query(query, conn)
        
        cursor.close()
        return df_resultados
        
    except Exception as e:
        print(f"Error: {e}")

def subir_modelo(modelo, municipio):
    try:
        # Conectarse a PostgreSQL
        conn = psycopg2.connect(
            dbname="punto_seguro", 
            user=DB_USER, 
            password=DB_PASSWORD, 
            host="pg-ed88bff-srodriguezcabana-914d.h.aivencloud.com", 
            port="23178"
        )
        conn.autocommit = True  # Para evitar commit explícitos
        cursor = conn.cursor()

        # Serializar el modelo usando pickle
        modelo_blob = BytesIO()
        pickle.dump(modelo, modelo_blob)
        modelo_blob.seek(0)  # Asegúrate de poner el puntero al principio del archivo

        # Descripción del modelo
        descripcion = f"Predicción Temperatura"

        # Insertar el modelo serializado en la base de datos
        cursor.execute("""
            INSERT INTO ps_app_modelosprediccion (
                modeloblob,
                descripcion,
                municipio
            ) VALUES (%s, %s, %s)
        """, (modelo_blob.read(), descripcion, municipio))

        print(f"Modelo de predicción para {municipio} guardado correctamente.")

    except Exception as e:
        print(f"Error al guardar el modelo: {e}")

    finally:
        cursor.close()
        conn.close()



## Consulta a la DB

In [2]:
query2="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('TEMPERATURA DEL AIRE A 2 m');
"""
data_temp = do_query(query2)

/tmp/ipykernel_86736/2250119809.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


In [4]:
data_temp

,fechaobservacion,valorobservado,departamento,municipio
0,2023-01-10 01:22:00+00:00,26.1,NORTE DE SANTANDER,CÚCUTA
1,2023-01-10 01:22:00+00:00,11.1,BOYACÁ,SOGAMOSO
2,2023-01-10 01:22:00+00:00,18.3,CUNDINAMARCA,GUAYABETAL
3,2023-01-10 01:22:00+00:00,28.4,ATLANTICO,SOLEDAD
4,2023-01-10 01:22:00+00:00,8.0,NARIÑO,ALDANA
...,...,...,...,...
5110663,2024-07-11 23:58:00+00:00,27.7,BOLIVAR,CARTAGENA DE INDIAS
5110664,2024-07-11 23:58:00+00:00,10.5,BOYACÁ,SOGAMOSO
5110665,2024-07-11 23:58:00+00:00,11.3,BOYACÁ,GUACAMAYAS
5110666,2024-07-11 23:58:00+00:00,22.7,CESAR,VALLEDUPAR


## Entrenamiento

In [45]:

# Cargar los datos
df = data_temp.copy()

# Convertir fecha a datetime
df['fechaobservacion'] = pd.to_datetime(df['fechaobservacion'])

# Extraer características adicionales
df['year'] = df['fechaobservacion'].dt.year
df['month'] = df['fechaobservacion'].dt.month
df['day'] = df['fechaobservacion'].dt.day
df['hour'] = df['fechaobservacion'].dt.hour

# Filtrar columnas necesarias para la predicción
features = ['year', 'month', 'day', 'hour']
target = 'valorobservado'

# Obtener la lista de municipios
municipios = df['municipio'].unique()

# Entrenar y almacenar un modelo por cada municipio
for municipio in municipios:
    print(f"Entrenando modelo para: {municipio}")

    # Filtrar datos por municipio
    datos_municipio = df[df['municipio'] == municipio]

    # Separar datos en entrenamiento y prueba
    X = datos_municipio[features]
    y = datos_municipio[target]

    if len(X) < 10:
        print(f"Insuficientes datos para {municipio}. Saltando.")
        continue

    # Dividir los datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Crear y entrenar el modelo
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    # Hacer una predicción
    y_pred = modelo.predict(X_test)

    # Mostrar las predicciones
    print("Predicciones:")
    print(y_pred)

    # Guardar el modelo en la base de datos
    subir_modelo(modelo, municipio)

    print(f"Modelo para {municipio} guardado en la base de datos.")


print("Entrenamiento y almacenamiento completado.")



Entrenando modelo para: CÚCUTA
Predicciones:
[28.94488954 25.44713842 28.99274268 ... 27.97723789 26.48732282
 28.99961044]
Modelo de predicción para CÚCUTA guardado correctamente.
Modelo para CÚCUTA guardado en la base de datos.
Entrenando modelo para: SOGAMOSO
Predicciones:
[12.60592704 14.13568142 13.75794161 ... 16.68333046 12.51215268
 15.70606809]
Modelo de predicción para SOGAMOSO guardado correctamente.
Modelo para SOGAMOSO guardado en la base de datos.
Entrenando modelo para: GUAYABETAL
Predicciones:
[21.83038844 21.21137882 19.45696951 ... 20.05611244 21.03564023
 22.4023822 ]
Modelo de predicción para GUAYABETAL guardado correctamente.
Modelo para GUAYABETAL guardado en la base de datos.
Entrenando modelo para: SOLEDAD
Predicciones:
[28.11287025 27.55099829 28.99089541 ... 27.16890101 27.44432974
 28.42198879]
Modelo de predicción para SOLEDAD guardado correctamente.
Modelo para SOLEDAD guardado en la base de datos.
Entrenando modelo para: ALDANA
Predicciones:
[12.67949174 1